# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [43]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [23]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [44]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [45]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [46]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [47]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [48]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [49]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [50]:
def get_links(url):
    website = Website(url)
    prompt = link_system_prompt + "\n\n" + get_links_user_prompt(website)

    # ارسال درخواست به سرور محلی Ollama
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llama3.2",  # یا "llama3" بسته به مدل دانلود شده
            "prompt": prompt,
            "stream": False
        }
    )

    # پردازش پاسخ و استخراج JSON
    if response.status_code == 200:
        response_data = response.json()
        model_output = response_data.get("response", "").strip()
        try:
            return json.loads(model_output)
        except json.JSONDecodeError:
            print("⚠️ JSON Decode Error: Model did not return valid JSON.")
            return {"links": []}
    else:
        print("⚠️ Error calling Ollama API:", response.status_code)
        return {"links": []}



In [53]:
result = get_links("https://anthropic.com")
print(json.dumps(result, indent=2))

{
  "links": [
    {
      "type": "about page",
      "url": "https://www.anthropic.com/company"
    },
    {
      "type": "careers page",
      "url": "https://www.anthropic.com/careers"
    },
    {
      "type": "news page",
      "url": "https://news.anthropic.com/"
    },
    {
      "type": "research page",
      "url": "https://www.anthropic.com/research"
    },
    {
      "type": "solutions page",
      "url": "https://anthropic.ai/solutions/coding"
    },
    {
      "type": "supported countries page",
      "url": "https://www.anthropic.com/supported-countries"
    }
  ]
}


In [41]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/perplexity-ai/r1-1776',
 '/deepseek-ai/DeepSeek-R1',
 '/microsoft/OmniParser-v2.0',
 '/stepfun-ai/stepvideo-t2v',
 '/stepfun-ai/Step-Audio-Chat',
 '/models',
 '/spaces/nanotron/ultrascale-playbook',
 '/spaces/lllyasviel/LuminaBrush',
 '/spaces/microsoft/OmniParser-v2',
 '/spaces/black-forest-labs/FLUX.1-dev',
 '/spaces/Steveeeeeeen/Zonos',
 '/spaces',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/facebook/natural_reasoning',
 '/datasets/SakanaAI/AI-CUDA-Engineer-Archive',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets/open-thoughts/OpenThoughts-114k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 

In [52]:
result = get_links("https://anthropic.com")
print(json.dumps(result, indent=2))

{
  "links": [
    {
      "type": "about page",
      "url": "https://www.anthropic.com/company"
    },
    {
      "type": "careers page",
      "url": "https://anthropic.com/careers"
    },
    {
      "type": "solutions page",
      "url": "https://anthropic.com/solutions/coding"
    },
    {
      "type": "research page",
      "url": "https://www.anthropic.com/research#entry:8@1:url"
    },
    {
      "type": "trust page",
      "url": "https://trust.anthropic.com/"
    },
    {
      "type": "legal/commercial-terms page",
      "url": "https://www.anthropic.com/legal/commercial-terms"
    },
    {
      "type": "news page",
      "url": "https://www.anthropic.com/news"
    }
  ]
}


## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [54]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [63]:
print(get_all_details("http://quotes.toscrape.com"))

Found links: {'links': [{'type': 'About page', 'url': 'http://quotes.toscrape.com/about'}, {'type': 'Company page', 'url': 'http://quotes.toscrape.com/'}, {'type': 'Careers/Jobs page', 'url': 'http://quotes.toscrape.com/careers'}]}
Landing page:
Webpage Title:
Quotes to Scrape
Webpage Contents:
Quotes to Scrape
Login
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
by
Albert Einstein
(about)
Tags:
change
deep-thoughts
thinking
world
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
by
J.K. Rowling
(about)
Tags:
abilities
choices
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
by
Albert Einstein
(about)
Tags:
inspirational
life
live
miracle
miracles
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
by
Jane Austen
(about)
Tags:
aliteracy
books
classi

In [65]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [66]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [68]:
print(get_brochure_user_prompt("ToScrap", "http://quotes.toscrape.com"))

Found links: {'links': [{'type': 'About page', 'url': 'https://www.goodreads.com/'}, {'type': 'Company page', 'url': 'https://www.zyte.com'}]}
You are looking at a company called: ToScrap
Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.
Landing page:
Webpage Title:
Quotes to Scrape
Webpage Contents:
Quotes to Scrape
Login
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
by
Albert Einstein
(about)
Tags:
change
deep-thoughts
thinking
world
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
by
J.K. Rowling
(about)
Tags:
abilities
choices
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
by
Albert Einstein
(about)
Tags:
inspirational
life
live
miracle
miracles
“The person, be it gentleman or lady, who has not pleasur

In [71]:
def create_brochure(company_name, url):
    website = Website(url)
    prompt = link_system_prompt + "\n\n" + get_brochure_user_prompt(company_name, url)

    # ارسال درخواست به Ollama
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": "llama3.2",  # اطمینان از وجود مدل
            "prompt": prompt,
            "stream": False
        }
    )

    # بررسی پاسخ
    if response.status_code == 200:
        response_data = response.json()
        print("Response JSON:", response_data)  # نمایش خروجی برای دیباگ

        model_output = response_data.get("response", "").strip()
        try:
            result_json = json.loads(model_output)
            display(Markdown(model_output))  # نمایش نتیجه در Markdown
            return result_json
        except json.JSONDecodeError:
            print("⚠️ JSON Decode Error: Model did not return valid JSON.")
            return {"links": []}
    else:
        print("⚠️ Error calling Ollama API:", response.status_code, response.text)
        return {"links": []}

In [76]:
create_brochure("python", "https://www.python.org")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.python.org/about/'}, {'type': 'company page', 'url': 'https://psf.org/'}, {'type': 'careers page', 'url': 'https://jobs.python.org/'}, {'type': 'community page', 'url': 'https://wiki.python.org/moin/community'}, {'type': 'blog page', 'url': 'https://python.org/dev/'}, {'type': 'news page', 'url': 'https://planetpython.org/'}]}
Response JSON: {'model': 'llama3.2', 'created_at': '2025-02-25T08:09:00.7539814Z', 'response': 'Here is a short brochure of the company "Python" in markdown, based on the provided webpage contents:\n\n# Welcome to Python.org\n\nPython is a high-level, interpreted programming language that aims to make it easy for developers to work quickly and integrate systems more effectively. With its clean syntax and indentation structure, Python is easy to learn and use, making it an ideal choice for both beginners and experienced programmers.\n\n## What We Do\n\n* Provide resources and documentation for Pyth

{'links': []}

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [77]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"


from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

def stream_brochure(company_name, url):
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [79]:
stream_brochure("ToScrap", "http://quotes.toscrape.com")

⚠️ JSON Decode Error: Model did not return valid JSON.
Found links: {'links': []}


# ToScrap Brochure

## Vision and Mission

At ToScrap, we believe that the power of knowledge is limitless. Our vision is to create intelligent systems that help people discover new ideas, grow, and explore. With a strong focus on innovation and customer satisfaction, we strive to be a leading provider of data scraping solutions.

## Quotes to Scrape

Our site is designed to inspire, educate, and entertain. We curate the most thought-provoking quotes from iconic philosophers, writers, and thinkers. From the wisdom of Einstein to the humor of Steve Martin, our quotes collection has something for everyone.

### Featured Authors

* Albert Einstein: "The world as we have created it is a process of our thinking."
* J.K. Rowling: "It is our choices, Harry, that show what we truly are."
* Jane Austen: "The person, be it gentleman or lady, who has not pleasure in a good novel..."
* Marilyn Monroe: "Imperfection is beauty, madness is genius..."

## Our Approach

At ToScrap, we understand the importance of adapting to change. We use cutting-edge AI and machine learning technologies to identify and collect valuable data from various sources.

### Made with ❤ by Zyte

Our solution is built on the backbone of Zyte's AI-powered NLP (Natural Language Processing) engine. This ensures that our quote scraping technology is fast, accurate, and scalable.

## Careers at ToScrap

Join us in shaping the future of intelligence. We're looking for talented individuals to join our team:

* Data Scientists: Help us develop intelligent systems for data collection and processing.
* NLP Engineers: Join our team of experts in Natural Language Processing.
* Developers: Contribute to the growth of our platform, helping it scale and improve.

## Tags

* change
* deep-thoughts
* thinking
* world
* inspirational
* life
* live
* miracle
* miracles
* readability
* humor

In [80]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'company page', 'url': 'https://huggingface.co/allenai'}, {'type': 'company page', 'url': 'https://huggingface.co/facebook'}, {'type': 'company page', 'url': 'https://huggingface.co/amazon'}, {'type': 'company page', 'url': 'https://huggingface.co/google'}, {'type': 'company page', 'url': 'https://huggingface.co/Intel'}, {'type': 'company page', 'url': 'https://huggingface.co/microsoft'}, {'type': 'company page', 'url': 'https://huggingface.co/grammarly'}, {'type': 'company page', 'url': 'https://huggingface.co/Writer'}, {'type': 'about page', 'url': 'https://ui.endpoints.huggingface.co'}]}


# Hugging Face: Building the Future of AI Together

At Hugging Face, we're dedicated to democratizing access to artificial intelligence (AI) by providing a unified platform for researchers and developers to collaborate on AI applications.

## Our Mission

Our mission is to build an inclusive community that fosters innovation in ML, making it more accessible and usable. We strive to make the latest advancements in ML available to everyone.

### Community Features

*   **Models**: Explore over 1 million pre-trained models for natural language processing (NLP), computer vision, and other tasks.
*   **Datasets**: Browse or create your own datasets, either public or private.
*   **Spaces**: Host unlimited public models, datasets, and applications to collaborate with others in our community.

## What We Offer

*   **Compute**: Deploy model-on-demand on our optimized inference endpoints without worrying about infrastructure management.
*   **Enterprise**: Get the most advanced platform for building AI solutions with enterprise-grade security, access controls, and dedicated support.
*   **Accelrate Your ML**: Our open-source stack allows you to move faster and explore various modalities, from text and images to video and audio.

As part of our commitment to innovation, we're working on key libraries such as:

### Open Source Projects 

*   **Transformers**: State-of-the-art AI for PyTorch and other Deep Learning frameworks.
*   **Diffusers**: Cutting-edge Diffusion models in Python
*   **Safetensors**: Safe way to store/distribute Neural Network Weights
*   **Hub Python Library**: Interact with our platform using a python client
*   **Tokenizers**: Fast, research and production-optimized tokenizers
*   **TRL**: Reinforcement Learning frameworks for transformers

## Our Values

At Hugging Face we strive to build a collaborative environment. We follow core values:

### Collaboration through AI 

*   Promote Accessibility and Inclusivity 
*   Foster Innovation through Knowledge Sharing 
*   Empower AI Researchers & Developers Worldwide
*   Build Communities Around Common Goals

Join us on our mission to revolutionize the future of AI.

[**Explore Our Models & Datasets »](#explore-ai-applications)
[**Discover Our Open Source Projects «](#open-source-projects]
**Get Started Today**

Whether you're a researcher, developer, or student, Hugging Face is here for you. Sign up for our platform and contribute to the future of AI together.

In [84]:
# imports

from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [85]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [86]:
# set up environment

load_dotenv()
openai = OpenAI()

In [87]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [88]:
# prompts

system_prompt = "You are a helpful technical tutor who answers questions about python code, software engineering, data science and LLMs"
user_prompt = "Please give a detailed explanation to the following question: " + question

In [89]:
# messages

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [ ]:
# Get gpt-4o-mini to answer, with streaming

stream = openai.chat.completions.create(model=MODEL_GPT, messages=messages,stream=True)
    
response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    response = response.replace("```","").replace("markdown", "")
    update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
# Get Llama 3.2 to answer

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))